In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
import xgboost

In [46]:
def transform_C():
    data_C = pd.read_excel("../data/江西省及各市碳排放总量.xlsx",index_col=0)
    data_nmy = pd.read_excel("../data/农牧业生产总值.xlsx",index_col=0)
    data_nlmyy = pd.read_excel("../data/农林牧渔业生产总值.xlsx",index_col=0)
    data_GDP = pd.read_excel("../data/GDP.xlsx",index_col=0)
    data_dqzrk = pd.read_excel("../data/地区总人口.xlsx",index_col=0)
    data_nczrk = pd.read_excel("../data/农村总人口.xlsx",index_col=0)
    with pd.ExcelWriter("../data/碳排放分解.xlsx") as writer:
        for col in data_C.columns:
            C_I = (data_C[col]/data_nmy[col]).values
            A_I = (data_nmy[col]/data_nlmyy[col]).values
            I_S = (data_nlmyy[col]/data_GDP[col]).values
            E_DL = (data_GDP[col]/data_dqzrk[col]).values
            U_RB = (data_dqzrk[col]/data_nczrk[col]).values
            P_R = data_nczrk[col]
            data = pd.DataFrame(C_I,columns=["C_I"],index=data_C.index)
            # print(C_I)
            data["A_I"] = A_I
            data["I_S"] = I_S
            data["E_DL"] = E_DL
            data["U_RB"] = U_RB
            data["P_R"] = P_R
            data["C"] = data_C[col]
            data.to_excel(writer,sheet_name=col)

transform_C()

In [47]:
def LMDI(data,tt,t0):
    def Delta_XX(yt,y0,xt,x0):
        def L(yt,y0):
            if yt == y0:
                return 0
            else:
                return (yt-y0)/(np.log(yt) - np.log(y0))
        return L(yt,y0)*np.log(xt/x0)
    Delta_y = data[tt,-1]-data[t0,-1]
    Delta_x = []
    for i in range(0,np.shape(data)[1]-1):
        Delta_x.append( Delta_XX(data[tt,-1],data[t0,-1],data[tt,i], data[t0,i]) )
    Delta_x.append(Delta_y)
    return Delta_x

In [49]:
def cal_LMDI():
    data_fj = pd.read_excel("../data/碳排放分解.xlsx",sheet_name=None,index_col=0)
    with pd.ExcelWriter("../data/碳排放LMDI.xlsx") as writer:
        for k,v in data_fj.items():
            data = v.values
            delta = []
            for year in range(1,len(data)):
                delta.append(LMDI(data,year,0))
            data_out = pd.DataFrame(np.array(delta),index=v.index[1:],columns=['Delta_'+col for col in v.columns])
            data_hj = pd.DataFrame(data_out.sum(),columns=["合计"]).T
            pd.concat([data_out,data_hj]).to_excel(writer,sheet_name=k)
        

cal_LMDI()